In [30]:
import pandas as pd
df = pd.read_csv('Suicide_Detection.csv', engine='python')

In [31]:
df = df.drop(columns=['Unnamed: 0'], axis=1)

In [32]:
df

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide
...,...,...
1482,help this person i think op has 2 days [https:...,suicide
1483,I'm riding a rollercoasterSo I recently starte...,suicide
1484,My dad has now started putting screen time on ...,non-suicide
1485,Well fuck anyone want to talk or something I’m...,non-suicide


In [33]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [34]:
!pip install torchtext

#Data Pre-processing

In [35]:
import re
import torch
import torch.nn as nn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
import torchtext.transforms as T
import spacy
from torchtext.vocab import build_vocab_from_iterator
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np

torch.manual_seed(1234)

def clean_text(text):
    text = re.sub(r'\S*@\S*\s?', '', text)
    text = re.sub(r'<.*?>', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = ''.join([char for char in text if char.isascii()])

    return text

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize(text):
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    return tokens

eng = spacy.load("en_core_web_sm")
def engTokenize(text):
    """
    Tokenize an English text and return a list of tokens
    """
    return [token.text for token in eng.tokenizer(text)]

def get_tokens(data_frame, column_name):
    for tokens in data_frame[column_name]:
        yield tokens

def getTransform(vocab):
    text_transform = T.Sequential(
        T.VocabTransform(vocab=vocab),
        T.AddToken(1, begin=True),
        T.AddToken(2, begin=False)
    )
    return text_transform


def apply_transform(sequence):
    transformed_sequence = getTransform(source_vocab)(sequence)
    return transformed_sequence

def applyPadding(sequence):
    """
    Convert a sequence of integers to a tensor and apply padding
    """
    return T.ToTensor(0)(sequence)


In [36]:
df['processed_text'] = df['text'].apply(clean_text)

In [37]:
df['processed_text'] = df['text'].apply(tokenize_and_lemmatize)

In [38]:
df['processed_text'] = df['text'].apply(engTokenize)

https://pytorch.org/tutorials/beginner/torchtext_custom_dataset_tutorial.html - torch.legacy.data has been depreciated, hence this is an alternative to pre-processing text using pytorch

In [39]:
source_vocab = build_vocab_from_iterator(
    get_tokens(df, 'processed_text'),
    min_freq=2, #skip words which frequency is more than 2
    specials=['<pad>', '<sos>', '<eos>', '<unk>'], #padding to make the shorter sentences equal to others for batch training
    special_first=True
)

source_vocab.set_default_index(source_vocab['<unk>'])

In [40]:
print(source_vocab.get_itos()[:9])

['<pad>', '<sos>', '<eos>', '<unk>', 'I', '.', ',', 'to', 'and']


In [41]:
temp_list = list(df['processed_text'])
some_sentence = temp_list[798][0]
print(some_sentence)
transformed_sentence = getTransform(source_vocab)(engTokenize(some_sentence))
print("Transformed sentence=", end="")
print(transformed_sentence)
index_to_string = source_vocab.get_itos()
for index in transformed_sentence:
    print(index_to_string[index], end=" ")

Secrets
Transformed sentence=[1, 3, 2]
<sos> <unk> <eos> 

In [52]:
df['class'].value_counts()

0    761
1    725
Name: class, dtype: int64

In [42]:
df['processed_text'] = df['processed_text'].apply(apply_transform)
df['processed_text'] = df['processed_text'].apply(applyPadding)
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])
df = df[df['class'] != 2]

In [43]:
X = df['processed_text'].tolist()
y = df['class'].tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
X_train_padded = pad_sequence(X_train, batch_first=True, padding_value=0)
X_test_padded = pad_sequence(X_test, batch_first=True, padding_value=0)

X_train_np = np.array(X_train_padded)
X_test_np = np.array(X_test_padded)
y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

X_train_tensor = torch.from_numpy(X_train_np)
X_test_tensor = torch.from_numpy(X_test_np)
y_train_tensor = torch.from_numpy(y_train_np)
y_test_tensor = torch.from_numpy(y_test_np)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
valid_data = TensorDataset(X_test_tensor, y_test_tensor)

In [45]:
batch_size = 20

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [46]:
dataiter = iter(train_loader)
sample_x, sample_y = next(dataiter)  # Correct way to get the first batch

print('Sample input size: ', sample_x.size())  # batch_size, seq_length
print('Sample input: \n', sample_x)

Sample input size:  torch.Size([20, 3505])
Sample input: 
 tensor([[   1,  821,  165,  ...,    0,    0,    0],
        [   1,    4,  847,  ...,    0,    0,    0],
        [   1, 1661,  330,  ...,    0,    0,    0],
        ...,
        [   1,  109,   76,  ...,    0,    0,    0],
        [   1, 4519,  107,  ...,    0,    0,    0],
        [   1,  821,  165,  ...,    0,    0,    0]])


#Model Training

In [47]:
class SentimentRNN(nn.Module):
    def __init__(self, no_layers, vocab_size, hidden_dim, embedding_dim, output_dim, drop_prob=0.5):
        super(SentimentRNN, self).__init__()

        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.no_layers = no_layers
        self.vocab_size = vocab_size

        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=self.hidden_dim,
                            num_layers=no_layers, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
      batch_size = x.size(0)
      # embeddings and lstm_out
      embeds = self.embedding(x)
      lstm_out, hidden = self.lstm(embeds, hidden)
      lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)

      # dropout and fully connected layer
      out = self.dropout(lstm_out)
      out = self.fc(out)

      # sigmoid function
      sig_out = self.sig(out)

      # reshape to be batch_size first
      sig_out = sig_out.view(batch_size, -1)
      sig_out = sig_out[:, -1]  # get last batch of labels

      # return last sigmoid output and hidden state
      return sig_out, hidden

    def init_hidden(self, batch_size):
        h0 = torch.zeros((self.no_layers, batch_size, self.hidden_dim))
        c0 = torch.zeros((self.no_layers, batch_size, self.hidden_dim))
        hidden = (h0, c0)
        return hidden


In [48]:
no_layers = 2
vocab_size = len(source_vocab) + 1
hidden_dim = 5
embedding_dim = 3
output_dim = 1
model = SentimentRNN(no_layers, vocab_size, hidden_dim, embedding_dim, output_dim)

#Model Evaluation - this may take a long time even I decreased the number of epoch.

In [54]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

lr = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

num_epochs = 1
train_losses = []
train_accuracies = []

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        hidden = model.init_hidden(inputs.size(0))
        outputs, _ = model(inputs, hidden)
        labels = labels.float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predictions = torch.round(outputs.squeeze())
        correct_predictions += torch.sum(predictions == labels.squeeze()).item()
        total_samples += labels.size(0)

    accuracy = correct_predictions / total_samples
    avg_loss = total_loss / len(train_loader)

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}')
    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)


KeyboardInterrupt: ignored